<a href="https://colab.research.google.com/github/Johyeyoung/DATA/blob/master/Figure_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 단어 리스트만들기

## 라이브러리 설치

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기
!pip install krwordrank # 키워드 추출기

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

## 데이터 가져오기

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/2-2._lesion_info_table.csv', encoding='cp949')

In [ ]:
# 같은 내용의 카테고리 합치기
df["lesion_detail_type"] = df["lesion_detail_type"].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)

# 3차 level의 종류
inspect_type=df['lesion_detail_type']

inspect_type=set(inspect_type.values.tolist())
print(inspect_type)
print(len(inspect_type))

{'USB포트_고장', '내비게이션_인터넷연결불가', '히터_고장', '주행_변속기이상', '히터_소음', '룸미러_파손', '주유카드_분실', '라이트/전구_경고등(고장)', '외관_누유', '창문_작동불가', '위생_쓰레기', '경고등_전/후측방 충돌 경고', '위생_구토', '후방카메라_파손(탈거, 각도이상)', '주차장_입차불가(만차)', '블랙박스_고장', '창문_썬팅', '충전카드_분실', '블랙박스_분실', '단말기_탈거', '시동_혼유', '주행_RPM이상', '에어컨_고장', '경고등_ESC(차세제어)', '브레이크_고장', '핸들_기능조작', '하이패스_카드오류', '외관_파손(하부)', '주행_변속레버파손', '도어_소음', '핸들_위치조절_고장', '와이퍼_파손', '주행_차량쏠림(얼라이먼트)', '경고등_온도게이지', '내장제_파손', '후방센서_고장', '선바이저_파손', '하이패스_고장', '주행_타는냄새', '경고등_DPF(배출가스)', '블랙박스_탈거', '오디오_버튼파손/분실', '시거잭_고장', '오디오_고장', '핸들_열선_고장', '단말기_신호불량', '번호판_분실', 'ADAS_고장', '오디오_블루투스', '워셔액_(통)파손', '단말기_파손', '위생_내부위생불량(모래, 흙, 먼지 등)', '오디오_AUX', '거치대_파손', '타이어_파스/펑크(후우)', '위생_담배(담배냄새, 담뱃재 등)', '사이드미러_고장', '전기차_충전커넥터_고장', '경고등_연료부족', '주차장_차량위치확인불가', '주유카드_비치함 파손/탈거', '본넷_고장', '주차장_환경이슈', '거치대_분실', '에어컨_소음', '핸들_소음', '주행_차량떨림', '오디오_소음', '주유구_(캡)분실', '도어_파손', '전기차_충전구 고장/파손', '트렁크_파손', '타이어_공기압경고등', '번호판_파손', '전기차_충전불가(태깅불가)', '글로브박스_파손', '목받침대_고장/분실', '창문_파손', '타이어_파스/펑크(전우)', '타이어_마모',

### 각 원인별 데이터

In [ ]:
def customer_info():
    phone_number='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*'
    url='(관제\s예약\s)*URL[주소:\s]*[^\n]+'
    location='(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*'
    lst = [phone_number, url, location]
    return '|'.join(lst)

def service_info():
    case_number='([원]*예약번호|블락번호)[^\n]*'
    card='(사고카드|장애카드|장애블락)[\s][^\n]*' # 장애카드 생성, 장애블락 생성
    picture='(사진[\s]*확인|사진[\s]*여부)[^\n]*' # 사진 여부 | 확인 
    customer='((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*'
    chatting='([\\n]*(채팅[\s]*상담|톡)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인 제거
    lst = [case_number, card, picture, customer, chatting]
    return '|'.join(lst)

In [ ]:
import re

dic_trigger={}
dic_figure={}

inspect_num=0
# lesion_detail_type 그룹별 데이터 확인
for inspect in inspect_type:
  inspect_num+=1
  #print(inspect_num)
  #if inspect_num<5: 
  trigger=[]
  figure=[]
  groupdf=df.groupby('lesion_detail_type').get_group(inspect) 
  lesion_id=groupdf['lesion_id']
  lesion_id=set(lesion_id.values.tolist())
  for id in lesion_id:
    groupdf2=groupdf.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    groupdf2=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf3=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'ㅁ', repl='\nㅁ\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(\n+)', repl=r'\n', regex=True)
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\s]', repl=r'', regex=True)
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(문의[\s]?구분[\s]*[아래\s중\s택1]*[\\n]*[ㄴ]*)|(문의[\s]?내용[\s]*[ㅁ|ㄴ]*)', repl=r'문의내용: ', regex=True)  # '문의 구분 택'의 경우
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[\s]*:[xXo0O/(완료)(확인불가) ]*', repl=r':', regex=True) # 여부 ox표시
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=customer_info(), repl=r'', regex=True)
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=service_info(), repl=r'', regex=True)
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'부위[\s]*:', repl=r'부위는 ', regex=True)
    # groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'긴출', repl=r'긴급출동', regex=True)

    memo=str(groupdf2['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger
    memo2=str(groupdf3['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger



    memo_list=memo.split('ㅁ')
    origin_list=memo2.split('ㅁ')
    #print(origin_list)
    #print(memo_list)
    factor_trgr = inspect.replace('_', '|') + '|고장|파손|문의|증상(\s+)상세'
    factor_trgr = re.compile(factor_trgr)
    factor_fig = re.compile(r'약속|안내|처리')
    flag = 0
    for i in memo_list:
      m = factor_trgr.search(i)
      p = factor_fig.search(i)
      if p:
        flag = 1
        figure.append(i)
        print('-------', i)
        figure_str=re.sub(r'\nㄴ|ㄴ', '\n', i)
        figure_str=re.sub(r'(\n+)', '\n', figure_str)
        figure_str=re.sub(r'[^\n]*(금액|비용|교환|환불|죄송|불편|쿠폰|항의|사과|종결|종료|종결)[^\n]*', '', figure_str)
        print(figure_str)
      else:
        if m and flag == 0:
          trigger.append(i)
          #print('+++++++', i)
        else:
          figure.append(i)
          print('-------', i)
    figure.append(groupdf2['memo'].values.tolist()[1:])
    figure_txt = str(groupdf2['memo'].values.tolist()[1:])  # 다음 원소가- Figure
    print("사후처리내용1:   ", figure_txt)
    figure_txt = "\n".join(groupdf2['memo'].values.tolist()[1:])  # 다음 원소가- Figure
    figure_txt=re.sub(r'[^\n]*(신규접수->처리중|처리중->조치완료)[^\n]*', '', figure_txt)
    figure_txt = str(figure_txt.split('\n'))
    print("사후처리내용2:   ", figure_txt, "\n\n")

  dic_trigger[inspect_num]=trigger
  dic_figure[inspect_num]=figure 
  break

------- 
확인/처리 내용
 - 장착물 종류  : 오디오/usb
 - 사진 확인 : X
 - 장애카드 생성 : O
 - 장애블락 생성 : X
 - 긴급출동 접수 : X


확인/처리 내용
 - 장착물 종류  : 오디오/usb
 - 사진 확인 : X
 - 장애카드 생성 : O
 - 장애블락 생성 : X
 - 긴급출동 접수 : X

------- 
 안내/약속 내용
ㄴ정보확인완료
ㄴ오디오최대 볼륨 높혔는데도 안되는지/안된다고함
ㄴ모드버튼으로 변경한건데도 안되는지/안된다고함
ㄴusb도안된다고하여 즉시 조치 어려워 반납후 점검요청해도될지/수긍/불편사과안내후 종결
※ 쏘카 장착물 장애는 긴급출동 불가함 (단말기 문제어 불가 제외)

 안내/약속 내용
정보확인완료
오디오최대 볼륨 높혔는데도 안되는지/안된다고함
모드버튼으로 변경한건데도 안되는지/안된다고함

※ 쏘카 장착물 장애는 긴급출동 불가함 (단말기 문제어 불가 제외)
사후처리내용1:    ['신규접수->처리중 (2021-01-27 16:28:22)', 'TS입고요청', '공기압점검 . 엔,와,향 교환 USB 정상작동 , 오디오관련 쏘카측 에서 업체 방문예정', '처리중->조치완료 (2021-01-28 15:20:46)']
사후처리내용2:    ['', 'TS입고요청', '공기압점검 . 엔,와,향 교환 USB 정상작동 , 오디오관련 쏘카측 에서 업체 방문예정', ''] 


------- 
확인/처리 내용
 - 장착물 종류  : 내비게이션 
 - 사진 확인 : X
 - 장애카드 생성 : O
 - 장애블락 생성 : X
 - 긴급출동 접수 : X


확인/처리 내용
 - 장착물 종류  : 내비게이션 
 - 사진 확인 : X
 - 장애카드 생성 : O
 - 장애블락 생성 : X
 - 긴급출동 접수 : X

------- 
 안내/약속 내용
 - 내비게이션 재시작으로 6번쨰 전화 주셨다고함
 - 배터리충전하려고했는데 USB 포트로 충전 불가하다고함
 - 불편드려서 너무 죄송하다고 안내
 - 배터리없는데 어떻게하는지
 -

In [ ]:
# 형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt
okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls



# 특수문자 제거
import re
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)



# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

In [ ]:
# csv 파일로 저장하기 
# import csv
# f = open('/content/drive/MyDrive/Trigger_list.csv','w', newline='\n')
# wr = csv.writer(f)
# wr.writerow(['type', '문의 내용'])



f = open('/content/drive/MyDrive/Trigger_list.txt','w')


# (사건 타입 번호 - 문의 사항) 저장
for idx, trigger in dic_trigger.items():
  trigger = set(trigger) # 1차 중복 제거
  
  # \n으로 나누기 전 문장 내에 \n으로 끊길 가능성 있는거 이어주기
  trigger_str= '\n'.join(trigger)
  trigger_str=re.sub(r'(\n+)', '\n', trigger_str)
  trigger_str=re.sub(r'\nㄴ|ㄴ', ' ', trigger_str)
  trigger_str=re.sub(r'(손상고장[\s][부위는]*|파손고장[\s][부위는]*|문의내용|문의사항|증상[\s]*상세내용|접수[\s]*상세[\s]*사유|접수내용|조치사항|확인사항|확인내용|현장조치)[:\s]*', '', trigger_str)
  trigger_str=re.sub(r'(여부[oOxX:]*)', '', trigger_str)
  trigger_str=re.sub(' +', ' ', trigger_str) # 다중 공백 제거
  trigger_str=re.sub(r'( )*\n( )*', '\n', trigger_str)
  trigger_str=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\n ]|(긴급출동 접수)\n', '', trigger_str) 
  trigger_str=re.sub('\n', '-', trigger_str) 

  #keyword_tracker2(trigger_str) # 형태소 추출기로 하면 정확도가 낮음 
  #trigger_str=correction(trigger_str)
  #print(trigger_str)
  # 맞춤법 검사기 돌리기 위해 특수문자 제거
  b = list(set(trigger_str.split('-')))

  print(sorted(b))
  b.sort(key=lambda x:len(x))
  #print(idx, b)


  for B in b:
    if len(B) > 3:
      #id\tdocument\tlabel\n : 형식
      # wr.writerow([idx, B]) # cs
      B=correction(B)
      print(B)
      f.write(str(idx)+'\t'+B+'\t'+str(idx)+'\n') # 각 리스트에 담긴 문장을 쓴다.

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
결제 시 결제가 되지 않아 장애 블록
주유카드 인식 불가로 고객 자비 결제
주유카드 인식 불가 주유 자비 결제
주유하러 왔는데 사용중지 카드라고 함
비즈니스 플랜 차량 주유카드 결제 오류건
주유카드가 사용중지 카드로 결제 오류 발생
주유하려 하는데 정지된 카드라고 나온다
주유카드 거래정지 카드라 나오신다며 문의
주유 카드 사용 정지로 나와 결제 불가 문의
대차 차량 주유카드 고장으로 결제 불가하여 인입
주유 중 주유카드 중지 카드라고 결제가 안되어 인입
주유카드로 결제시도 시 사용중지 카드로 나와 인입
주유카드 결제 시 승인되지 않는 카드라고 출력
차량 운행 중인데 주유카드가 결제 거절 카드로 문의
차량 내 주유카드가 이용 중지 카드로 나온다고 하심
주유하려고 하는데 주유카드가 정지된 카드로 나온다고 하심
주유카드가 안된다고 개인카드로 결제했다고 환불 요청
주유카드로 결제 시 한도 초과로 결제 불가하다고 유입
주유소 군데 방문했으나 유효한 카드가 아니라며 인식되지 않음
결제 시 셀프 주유기 오류 출력 신용 승인 오류라는 내용 표시됨
주유하려고 하는데 주유카드 정지 카드라고 확인된다고 하여 인입
주유카드로 결제 불가하여 주유하지 못하고 반납하여 페널티 발생
반납한 차량 카드 IC 칩에 이상이 있어 주유가 밖에 되지 않기에 문의
고객 이용 시 주유카드 두 개가 있었으나 한 장은 이용불가였고 한 장은 분실카드였다고 함
차량 이용하려고 탔는데 기름이 없어서 주유하려고 하니까 카드 ic 오류라고 계속 결제가 안됨
주유카드 결제 불가로 사비 결제 후 환불 접수하였으나 내역 확인 불가로 반려됐다고 하심
주유 결제 만원 진행 후 원만 주유됨 취소를 진행했으나 원 재결제 진행시 결제가 안되어 귀가를 못하고 계신다 함 재결제 시 오류 메시지 확인됨
['', '계속 업데이트와 재부팅을 반복합니다', '고객님 네비게이션 불량인입', '고객님 네비게이션 업데이트불량 사용불가인입', '고객님 네비게이션 업테이트오류 후방카

In [ ]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
def keyword_tracker(trigger):
  texts = [normalize(text, english=True, number=True) for text in trigger]  # 문장내 특수문자 제거하기
  texts
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )

  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10

  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
  return

In [ ]:
def keyword_tracker2(memo):
  memo=sub_special(memo) # memo-문자열
  memo=morph_and_stopword(memo)
  memo=Counter(memo)
  memo_rank=memo.most_common(100)
  print(memo_rank)
  return

description=str(set(groupdf['description'].values.tolist())) 
description=sub_special(description)
description=morph_and_stopword(description)
description=Counter(description)
description_rank=description.most_common(100)


print(description_rank)
# 형태소 분리
#print('memo:\n', morph_and_stopword(memo))
#print('description:\n', morph_and_stopword(description))


## 객체인식명


In [ ]:
!git clone https://github.com/eagle705/pytorch-bert-crf-ner.git

In [ ]:
!pip3 install torch torchvision
!pip3 install pytorch_pretrained_bert>=0.4.0
!pip3 install mxnet>=1.5.0
!pip3 install gluonnlp>=0.6.0
!pip3 install sentencepiece>=0.1.6
!pip3 install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
!pip3 install transformers
!pip3 install tb-nightly
!pip3 install future
import os


In [ ]:
!pip3 install Config
!pip3 install Tokenizer

In [ ]:
!python pytorch-bert-crf-ner/inference.py
